# Samples 04-kernel-arguments-chat

Source : <https://github.com/microsoft/semantic-kernel/blob/main/dotnet/notebooks/04-kernel-arguments-chat.ipynb>

In [1]:
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 1.13.0"

Installed Packages Microsoft.SemanticKernel, 1.13.0

## Instantiate the kernel


In [2]:
// Load some helper functions, e.g. to load values from settings.json
#!import config/Settings.cs
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;

//Create Kernel builder
var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

Console.WriteLine($"Use Azure Open AI : {useAzureOpenAI}");
Console.WriteLine($"Model : {model}");
Console.WriteLine($"Azure EndPoint : {azureEndpoint}");
// Console.WriteLine($"API Key : {apiKey}");

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Use Azure Open AI : True
Model : gpt-4o
Azure EndPoint : https://oai-ygo74-switzerland.openai.azure.com/


In [3]:
const string skPrompt = @"
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$userInput}}
ChatBot:";

var executionSettings = new OpenAIPromptExecutionSettings
{
    MaxTokens = 2000,
    Temperature = 0.7,
    TopP = 0.5
};

In [4]:
var chatFunction = kernel.CreateFunctionFromPrompt(skPrompt, executionSettings);

In [5]:
var history = "";
var arguments = new KernelArguments()
{
    ["history"] = history
};

In [6]:
var userInput = "Hi, I'm looking for book suggestions";
arguments["userInput"] = userInput;

var bot_answer = await chatFunction.InvokeAsync(kernel, arguments);

In [7]:
history += $"\nUser: {userInput}\nAI: {bot_answer}\n";
arguments["history"] = history;

Console.WriteLine(history);


User: Hi, I'm looking for book suggestions
AI: Hi! I'd be happy to help with book suggestions. Could you tell me a bit about what genres or types of books you enjoy? For example, do you like fiction, non-fiction, mystery, fantasy, science fiction, romance, or something else?



In [8]:
Func<string, Task> Chat = async (string input) => {
    // Save new message in the arguments
    arguments["userInput"] = input;

    // Process the user message and get an answer
    var answer = await chatFunction.InvokeAsync(kernel, arguments);

    // Append the new interaction to the chat history
    var result = $"\nUser: {input}\nAI: {answer}\n";
    history += result;

    arguments["history"] = history;

    // Show the response
    Console.WriteLine(result);
};

In [9]:
await Chat("I would like a non-fiction book suggestion about Greece history. Please only list one book.");


User: I would like a non-fiction book suggestion about Greece history. Please only list one book.
AI: Sure! I recommend "The Histories" by Herodotus. It's a classic work that provides a detailed account of ancient Greece and its surrounding regions, written by the ancient Greek historian Herodotus. This book is often considered one of the foundational works of history in Western literature.



In [10]:
await Chat("that sounds interesting, what are some of the topics I will learn about?");


User: that sounds interesting, what are some of the topics I will learn about?
AI: In "The Histories" by Herodotus, you will learn about a wide range of topics related to ancient Greece and its neighboring regions. Some of the key topics include:

1. **The Greco-Persian Wars**: Detailed accounts of the conflicts between the Greek city-states and the Persian Empire.
2. **Cultural Practices**: Descriptions of the customs, traditions, and daily life of various peoples, including the Egyptians, Persians, and Scythians.
3. **Geography and Ethnography**: Insights into the geography of the ancient world and the diverse cultures that inhabited it.
4. **Political History**: Information about the political systems, rulers, and significant events in different regions.
5. **Mythology and Legends**: Stories and legends that were part of the oral traditions of various cultures.
6. **Economics and Trade**: Discussions on the economic activities, trade routes, and resources of different regions.
7. *

In [11]:
await Chat("Which topic from the ones you listed do you think most people find interesting?");


User: Which topic from the ones you listed do you think most people find interesting?
AI: While interest can vary greatly from person to person, many readers find the accounts of the **Greco-Persian Wars** particularly fascinating. These sections provide detailed narratives of key battles such as the Battle of Marathon, the Battle of Thermopylae, and the Battle of Salamis. The stories of heroism, strategy, and the clash of civilizations often captivate readers and offer a dramatic glimpse into the ancient world. Additionally, the interplay between the relatively small Greek city-states and the vast Persian Empire adds a compelling dynamic to the historical narrative.



In [12]:
await Chat("could you list some more books I could read about the topic(s) you mentioned?");


User: could you list some more books I could read about the topic(s) you mentioned?
AI: Certainly! Here are some more books that delve into the topics mentioned, particularly focusing on the Greco-Persian Wars and related subjects:

1. **"The Persian Wars" by Herodotus** - This is a more focused edition of Herodotus's work, specifically concentrating on the Greco-Persian Wars.

2. **"The Landmark Herodotus: The Histories" edited by Robert B. Strassler** - This edition includes extensive maps, annotations, and appendices to help readers better understand Herodotus's work.

3. **"The Greco-Persian Wars" by Peter Green** - A modern and accessible account of the wars between Greece and Persia, providing context and analysis.

4. **"The Battle of Salamis: The Naval Encounter that Saved Greece—and Western Civilization" by Barry Strauss** - A detailed and engaging account of the pivotal naval battle during the Greco-Persian Wars.

5. **"Persian Fire: The First World Empire and the Battle for

In [13]:
Console.WriteLine(history);


User: Hi, I'm looking for book suggestions
AI: Hi! I'd be happy to help with book suggestions. Could you tell me a bit about what genres or types of books you enjoy? For example, do you like fiction, non-fiction, mystery, fantasy, science fiction, romance, or something else?

User: I would like a non-fiction book suggestion about Greece history. Please only list one book.
AI: Sure! I recommend "The Histories" by Herodotus. It's a classic work that provides a detailed account of ancient Greece and its surrounding regions, written by the ancient Greek historian Herodotus. This book is often considered one of the foundational works of history in Western literature.

User: that sounds interesting, what are some of the topics I will learn about?
AI: In "The Histories" by Herodotus, you will learn about a wide range of topics related to ancient Greece and its neighboring regions. Some of the key topics include:

1. **The Greco-Persian Wars**: Detailed accounts of the conflicts between the G